In [1]:
import numpy as np
import scipy as sp
import sympy as sym

Given the following functional: 

$$J(u) = \int_{\Omega \subset \mathbb{R}} L(x, u, u_x) \ dx  = \int_{\Omega \subset \mathbb{R}}(u(x)-f_0(x))^2 + \alpha \ \Psi_1(u(x)-f_0(x)) + \beta \ \Psi_2(|u'(x)|) \ dx $$

then when we try to minimize it, the corresponding Euler-Lagrange equation becomes:

$$\frac{\partial L}{\partial u}-\frac{d}{dx}\frac{\partial L}{\partial u_x} =  2(u(x)-f_0(x)) + \alpha \ \Psi_1'(u(x)-f_0(x)) - \beta \ u_{xx}(x) \ \Psi_2''(|u_{x}(x)|)$$

In [2]:
#setting parameters
a = 0.1
b = 1.
N = 50

z = sym.Symbol('z')

#Penalizing function and its derivatives
psi1 = sym.exp(z)
d1psi1 = sym.diff(psi1, z)
d1psi1 = sym.lambdify(z, d1psi1, modules='numpy')

#Smoothing function and its derivatives
lamb = 0.5
psi2 = 2*lamb*sym.log(1 + z**2/lamb)
d1psi2 = sym.diff(psi2, z)
d2psi2 = sym.diff(d1psi2, z)
d1psi2 = sym.lambdify(z, d1psi2, modules='numpy')
d2psi2 = sym.lambdify(z, d2psi2, modules='numpy')

In [3]:
"""
RBF functions and its derivatives
"""
def phi(x, x0, c, sig):
    return c * np.exp(-(x-x0)**2/(2*sig**2))

def phix(x, x0, c, sig):
    return -(c/sig**2) * np.exp(-(x-x0)**2/(2*sig**2)) * (x-x0)

def phixx(x, x0, c, sig):
    return (c/sig**4) * np.exp(-(x-x0)**2/(2*sig**2)) * ((x-x0)**2 - sig**2)

In [4]:
def F(X):
    #unpacking parameters
    N = len(X)/2
    c = X[0:N]
    sig = X[N:]
    return

#wrapper function
def F(X,t):
    return F(X)